In [1]:
# テキストファイルの先頭行を表示する関数
# fname: テキストファイル名
# n=: 表示する行数
# encoding=: テキストのエンコーディング(cp932,shift_jis,utf-8(デフォルト)など)
def head(fname,n=10,encoding='utf-8'):
    with open(fname,encoding=encoding) as f:
        for _ in range(n):
            print(f.readline(),end='')

In [2]:
# コード 2.1 Webからのファイルダウンロード

import urllib.request  #urllibライブラリをインポート

#ダウンロードファイルのURLは事前に確認する
targetURL='http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Japan_3_Factors_Daily_CSV.zip'
regFileName='jpn3factor.zip'   #保存ファイル名

#urllib.request.urlretrieve()メソッドでファイルをダウンロードする
#  対象ファイルのURL（第1引数）,保存するファイル名（第2引数）
urllib.request.urlretrieve(targetURL,regFileName)

('jpn3factor.zip', <http.client.HTTPMessage at 0x10d9a6cd0>)

In [3]:
# コード 2.2 zipファイルの解凍
#   先行実行コード：コード2.1

import zipfile  #zipfileライブラリをインポート

zipFileName='jpn3factor.zip'  #解凍対象のzipファイル名（ここではコード 2.1でダウンロードしたファイル）
outDirName='./'  #解凍先のディレクトリ名
outFileName='Japan_3_Factors_Daily.csv'  #解凍後のファイル名

with zipfile.ZipFile(zipFileName) as zf: # zipファイルオープン
    #zipファイルの中に格納されているファイル名を一覧表示
    print(zf.namelist())
    # ['Japan_3_Factors_Daily.csv']

    #すべてのファイルを指定ディレクトリへ解凍
    # （指定ディレクトリが存在しない場合は自動生成される）
    zf.extractall(outDirName)

    #特定ファイル（第1引数）を指定ディレクトリ（第2引数） へ解凍
    zf.extract(outFileName, outDirName)

    #パスワード付きzipファイルの場合はpwd引数にパスワードを指定
    password=b'qwerty'  #パスワード値はバイト型で指定
    zf.extract(outFileName, outDirName, pwd=password)

#解凍したファイル内容の表示（先頭10行）
head(outDirName+outFileName)


['Japan_3_Factors_Daily.csv']
This file was created using the 202103 Bloomberg database.

Missing data are indicated by -99.99.



,Mkt-RF,SMB,HML,RF
19900702    ,0.85    ,0.38   ,-0.06    ,0.03
19900703    ,0.07    ,0.72    ,0.30    ,0.03
19900704    ,1.45    ,0.52    ,0.26    ,0.03


In [4]:
# コード 2.3 Webページの取得

import urllib  #urllibライブラリをインポート

#取得するWebページのURLは事前に確認する
targetURL='https://www.kantei.go.jp/jp/rekidainaikaku/index.html'
regFileName='rekidainaikaku.html'  #保存ファイル名

#urllib.request.urlretrieve()メソッドでWebページ（HTMLデータ）をダウンロードする
#  対象ファイルのURL（第1引数）,保存するファイル名（第2引数）
urllib.request.urlretrieve(targetURL,regFileName)

#取得したデータ内容の表示（先頭10行）
head(regFileName)


﻿<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="description" content="歴代内閣の情報をご覧になれます。">
    <meta name="keywords" content="首相官邸,政府,内閣,総理,歴代内閣">
    <meta property="og:title" content="歴代内閣 | 首相官邸ホームページ">
    <meta property="og:type" content="article">


In [5]:
# コード 2.4 HTMLファイルからのデータ取得
#   先行実行コード：コード2.3

from bs4 import BeautifulSoup  #BeautifulSoupライブラリをインポート

inFileName='rekidainaikaku.html'  #対象のHTMLファイル（ここではコード 2.3で取得したHTMLファイル）
outFileName='rekidainaikaku.txt'  #出力ファイル（対象HTMLを解析して出力したもの）

#解析対象のhtmlファイルをオープン
with open(inFileName, mode='r', encoding='utf-8') as f:
  #htmlファイルの内容をBeautifulSoupパーサーに読み込ませる
  soup = BeautifulSoup(f.read(), 'html.parser')

  #出力用ファイルのオープン
  with open(outFileName, mode='w', encoding='utf-8') as fo:
    #titleタグの文字列を取得
    _title = soup.find('title').get_text()
    print(_title, file=fo)
    #h3タグ内の文字列を取得
    _names = [_h3.get_text() for _h3 in soup.find_all('h3')]
    print('\n'.join(_names), file=fo)

#取得したデータ内容の表示（先頭10行）
#Windowsではopenして書き込むとcp932エンコーディング(sjis)になる。
head(outFileName)


歴代内閣 | 首相官邸ホームページ
第98代安倍 晋三
第97代安倍 晋三
第96代安倍 晋三
第95代野田 佳彦
第94代菅 直人
第93代鳩山 由紀夫
第92代麻生 太郎
第91代福田 康夫
第90代安倍 晋三


In [6]:
# コード 2.5 WebAPIデータの取得

# NewsAPIを使ってニュースデータをダウンロードする
#################################################################
# news apiのキーを指定しないとエラーとなるので注意すること。
#################################################################
import urllib.parse
import urllib.request

# news apiを使ったニュースのダウンロード(https://newsapi.org/) 2020/07現在
#   Developer plan
#   検索期間：１ヶ月前〜1時間前
#   リクエスト数：500/日
# params (辞書で与える) 以下は指定可能な主なパラメータ(https://newsapi.org/docs/endpoints/everything#sources)
#   apiKey: 個人で取得したnews apiのapi key(必須)
#   q: titleとbodyのキーワードやフレーズ検索, AND/OR/NOTが使える ex. (ビール OR ワイン) AND 夏
#   qInTitle: タイトルのみを検索する。書式はqに同じ
#   language: 言語 (en,jpなど) デフォルトは全言語
#   from,to: 検索期間(デフォルト: 契約planの最古と最新日) ex. from=2020-07-09
#   pageSize: 取得する記事数
#   domains: 取得する記事元のURLのドメイン(カンマで区切って複数指定可能)
#   sortBy: relevancy, popularity, publishedAt(デフォルト)
def downloadNews(params,jsonFile):
    url='http://newsapi.org/v2/everything'
    requestStr=url+'?'+ urllib.parse.urlencode(params)
    print(requestStr)
    urllib.request.urlretrieve(requestStr,jsonFile)
    
params = {'language':'jp',
          'pageSize':50,
          'qInTitle':'株価',
          'domains': 'yahoo.co.jp',
          'apiKey':'### 個人で取得したnews apiのキーを指定する ###'
         }
downloadNews(params,'./stock_yahoo.json')
#################################################################
# 最新のニュースタイトルを取得しているため、「株価」を含むタイトルが少ないこともありうる。
# 出力された./stock_yahoo.jsonの中身を確認し、件数が少なければ、最近流行りの言葉に置き換えて検索されたい。
#################################################################

#出力ファイル内容の表示（先頭2000文字）
with open('./stock_yahoo.json',encoding='utf-8') as f:
    for i,c in enumerate(f.read()):
        print(c,end='')
        if i>2000:
            break


http://newsapi.org/v2/everything?language=jp&pageSize=50&qInTitle=%E6%A0%AA%E4%BE%A1&domains=yahoo.co.jp&apiKey=xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
{"status":"ok","totalResults":1,"articles":[{"source":{"id":null,"name":"Yahoo.co.jp"},"author":"オーサー一覧","title":"血栓症の副反応、変異株への効果低下　5月承認予定のアストラゼネカ社の新型コロナワクチン 現時点の評価(忽那賢志)","description":"日本国内でも5月にも承認される見込みとされるアストラゼネカ社の新型コロナワクチンに血栓症の副反応が報告されました。副反応の報告の詳細や変異株に対する有効性低下など、現時点での評価についてまとめました。","url":"https://news.yahoo.co.jp/byline/kutsunasatoshi/20210418-00233281/","urlToImage":"https://newsbyl-pctr.c.yimg.jp/r/iwiz-yn/rpr/kutsunasatoshi/00233281/title-1618732976941.jpeg","publishedAt":"2021-04-18T08:23:00Z","content":"25\r\nCDC \"How Viral Vector COVID-19 Vaccines Work\"\r\n/ChAdOx1 nCoV-19AZD1222\r\nChAdOx1 nCoV-19\r\n2\r\n3https://doi.org/10.7326/M21-0111\r\n2\r\n14122WHO812\r\n70.4mRNA95%\r\nmRNA28mRNA\r\nWHO situation report 13 Apr… [+101 chars]"}]}

In [7]:
# コード 2.6 JSONファイルの入出力
#   先行実行コード：コード2.5

import json  #jsonライブラリをインポート

#入力JSONファイル名（ここではコード 2.5で取得したJSONファイル）
inFileName='stock_yahoo.json'
#出力JSONファイル名
outFileName='stock_yahoo_mini.json'

#対象のJSONファイルをオープン
with open(inFileName, mode='r', encoding='utf-8') as f:
    #json.load()関数によりJSONファイルを辞書型変数に変換
    jsonDict = json.load(f)

    outLst = []   #保存用のリスト変数
    for item in jsonDict['articles']:
        outLst.append({
            'title': item['title'],
            'author': item['author']
        })

    #保存用のJSONファイルをオープン
    with open(outFileName, mode='w',encoding='utf-8') as fo:
        #json.dump()関数によりJSONファイルとして保存
        #  第1引数:保存する変数, 第2引数:保存するファイル名,
        #     indent引数:整形時の字下げ数,
        #     ensure_ascii引数:False指定で日本語の全角文字などをそのまま出力
        json.dump(outLst, fo, indent=4, ensure_ascii=False)
        
#出力JSONファイル内容の表示（先頭20行）
head(outFileName,n=20)


[
    {
        "title": "血栓症の副反応、変異株への効果低下　5月承認予定のアストラゼネカ社の新型コロナワクチン 現時点の評価(忽那賢志)",
        "author": "オーサー一覧"
    }
]

In [8]:
# コード 2.7 ディレクトリにあるファイルリストの抽出

import glob  #globライブラリをインポート
import os  #osライブラリをインポート（パス結合で使用するため）

#対象ディレクトリ
dir_in = 'in/stats/2018/'

#glob.glob()関数の引数に対象ディレクトリのパスを指定することで、ディレクリトリ内に存在するファイルリストを取得できる。
#ここでは、'in/stats/2018/*.csv'というパスを指定しており、*の箇所に任意の文字が含まれるファイルのパスが抽出される。
#os.path.join()関数の引数にディレクトリ名とファイル名を指定するとパスの形式で結合してくれる。
for f in glob.glob(os.path.join(dir_in, '*.csv')):
    print(f)
    # in/stats/2018/a020047.csv
    # in/stats/2018/a020013.csv
    # in/stats/2018/a020001.csv
    # in/stats/2018/a020014.csv

    #各ファイルの先頭5行がCSV形式以外の説明行であるため、
    #先頭5行を削除したCSV形式ファイルを別名で作成
    with open(f+'_after', mode='w',encoding='utf-8') as f_out:
        with open(f, mode='r',encoding='cp932') as f_in:
            f_out.write('\n'.join(f_in.read().split('\n')[5:]))

#入力ファイルの一つ（in/stats/2018/a020001.csv）の内容表示（先頭10行）
#このファイルはダウンロードした大元のエンコーディングがcp932である。
print('\n■入力ファイルの一つ（in/stats/2018/a020001.csv）の内容表示（先頭10行）')
head('in/stats/2018/a020001.csv',encoding='cp932')

#出力ファイルの一つ（in/stats/2018/a020001.csv_after）の内容表示（先頭10行）
#出力のエンコーディングは上のopen関数でutf-8を指定している。
print('\n■出力ファイルの一つ（in/stats/2018/a020001.csv_after）の内容表示（先頭10行）')
head('in/stats/2018/a020001.csv_after')


in/stats/2018/a020047.csv
in/stats/2018/a020013.csv
in/stats/2018/a020001.csv
in/stats/2018/a020014.csv

■入力ファイルの一つ（in/stats/2018/a020001.csv）の内容表示（先頭10行）
平成３０年,人口動態統計,,
中巻　総覧　第２表－０１　人口動態総覧（件数），都道府県（北海道）・市部－郡部－保健所－市区町村別
,出生数,（再掲）,死亡数,（再掲）, ,死産数, ,周産期死亡, , ,婚姻件数,離婚件数
,　　　,2500g未満,　　　,乳児死亡数,新生児,自然死産,人工死産,総　数,22週以後,早期新生児,　　　,　　　
,,,,,死亡数,,,,の死産数,死亡数,,
０１北海道　　,32642,2987,64187,62,32,388,493,118,95,23,22916,9971
　　札幌市　　,13248,1182,19343,34,20,138,208,47,32,15,9878,4024
　　その他の市,14104,1371,30851,23,10,182,216,54,48,6,9665,4459
　　郡　部　　,5290,434,13993,5,2,68,69,17,15,2,3373,1488
0110札幌市　　　　　　　,13248,1182,19343,34,20,138,208,47,32,15,　　　　...,　　　　...

■出力ファイルの一つ（in/stats/2018/a020001.csv_after）の内容表示（先頭10行）
０１北海道　　,32642,2987,64187,62,32,388,493,118,95,23,22916,9971
　　札幌市　　,13248,1182,19343,34,20,138,208,47,32,15,9878,4024
　　その他の市,14104,1371,30851,23,10,182,216,54,48,6,9665,4459
　　郡　部　　,5290,434,13993,5,2,68,69,17,15,2,3373,1488
0110札幌市　　　　　　　,13248,1182,19343,34,20,138,208,47,32,15,　　　　...,　　　　..

In [9]:
# コード 2.8 nysol_pythonを利用したCSVファイルの入出力

# ※Windows環境下ではnysol_pythonは動作させることができません。

import nysol.mcmd as nm  #nysol.mcmdライブラリをインポート

#入力CSVファイル名（ここではコード 2.7で生成したファイル群のワイルドカード表現）
inFileName='in/stats/2018/*.csv_after'
#出力CSVファイル名
outFileName='nysol_python.csv'

#MCMDコマンド格納変数を宣言
mcmd=None
#mcatメソッドはファイルの読み込み・統合ができる
#msortfメソッドは指定項目で行の並び替えができる
#  i=引数：入力CSVファイル名を指定
#  nfn=True：先頭行をタイトル行とみなさない
#  msortfでのf=引数：ソートの項目キーを指定（0項目）
#  o=引数：出力CSVファイル名を指定
#<<=記述子により処理メソッドを順番に格納
mcmd<<=nm.mcat(i=inFileName, nfn=True)
mcmd<<=nm.msortf(nfn=True, f='0', o=outFileName)
#run()メソッドにより格納した処理メソッドが順番に実行される
mcmd.run()

#出力ファイル内容の表示（先頭5行）
head(outFileName,n=5)


01101札幌市中央区　　,1744,170,2174,6,5,21,58,6,4,2,1556,537
01102札幌市北区　　　,1847,163,2842,3,1,20,17,8,7,1,1213,601
01103札幌市東区　　　,1978,163,2512,5,4,17,37,5,2,3,1408,580
01104札幌市白石区　　,1711,163,2032,4,1,17,18,4,3,1,1455,541
01105札幌市豊平区　　,1682,130,2047,4,2,16,21,7,5,2,1458,445


In [10]:
# コード 2.9 pandasを利用したCSVファイルの入出力

import pandas as pd  #pandasライブラリをインポート

#入力CSVファイル名
inFileName='in/ds2qpr.csv'

#出力CSVファイル名
outFileName='pandas.csv'
#CSVファイルの読み込み
df_file = pd.read_csv(inFileName)

#正常に読み込めているか、内容確認
print(df_file.head(3))

#一部の項目のみ抽出してCSVファイルとして出力
df_file[["モニタ","日付","購入数量","都道府県","小分類名"]].to_csv(outFileName)

#入力ファイル内容の表示（先頭10行）
print('\n■入力ファイル内容の表示（先頭10行）')
head(inFileName)

#出力ファイル内容の表示（先頭10行）
print('\n■出力ファイル内容の表示（先頭10行）')
head(outFileName)


   モニタ        日付 購入先区分送信ID 店舗  業態   商品  購入数量     単価     金額  都道府県  ... 中高生有無  \
0  00J  20140120       NaN  2   2  6JN     1  248.0  248.0     0  ...     0   
1  00J  20140119      0CjU  D   1   u9     1   98.0   98.0     0  ...     0   
2  00J  20140201       NaN  u   5  fEw     2  198.0  396.0     0  ...     0   

   大人有無  老人有無  曜日  大分類名  中分類名    小分類名         細分類名      店舗名          業態名  
0     1     0   月    食品  加工食品   漬物・佃煮           漬物  セブンイレブン   コンビニエンスストア  
1     1     0   日    食品  生鮮食品      農産        その他農産     いなげや         スーパー  
2     1     0   土   日用品  日用雑貨  住居用洗剤類  使い捨て紙クリーナー類  マツモトキヨシ  薬粧店・ドラッグストア  

[3 rows x 31 columns]

■入力ファイル内容の表示（先頭10行）
モニタ,日付,購入先区分送信ID,店舗,業態,商品,購入数量,単価,金額,都道府県,メーカー,大分類,中分類,小分類,細分類,性別,年代,未既婚,メイン買物担当者,乳幼児有無,小学生有無,中高生有無,大人有無,老人有無,曜日,大分類名,中分類名,小分類名,細分類名,店舗名,業態名
00J,20140120,,2,2,6JN,1,248,248,0,3z,1,11,1117,111701,1,10,1,1,0,0,0,1,0,月,食品,加工食品,漬物・佃煮,漬物,セブンイレブン,コンビニエンスストア
00J,20140119,0CjU,D,1,u9,1,98,98,0,fo,1,12,1203,120397,1,10,1,1,0,0,0,1,0,日,食品,生鮮食品,農産,

In [11]:
# コード 2.10 EXCELから表データを取得する例

import pandas as pd  #pandasライブラリをインポート

#入力EXCELファイル名
inFileName='./in/1807nsjin.xls'
#出力CSVファイル名
outFileName='pandas2excel.csv'
# 第1引数にCSVファイル名を指定、その他必要な引数を指定
df = pd.read_excel(
    inFileName,
    sheet_name='人口、世帯数、人口動態（市区町村別）【日本人住民】',
    header=1
)
df.to_csv(outFileName)

#出力ファイル内容の表示（先頭10行）
head(outFileName)


,団体コード,都道府県名,市区町村名,平成30年,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,平成29年,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,,,,人口,,,世帯数,,,住民票記載数,,,,,,,,住民票消除数,,,,,,,,増減数(Ａ)-(Ｂ),増減率,自然増減数,自然増減率,社会増減数,社会増減率
1,,,,男,女,計,日本人住民,複数国籍,計,転入者数（国内）,転入者数（国外）,転入者数（計）,出生者数,その他（帰化等）,その他（その他）,その他（計）,計（Ａ）,転出者数（国内）,転出者数（国外）,転出者数（計）,死亡者数,その他（国籍喪失）,その他（その他）,その他（計）,計（Ｂ）,,,,,,
2,,合計,,61098245,64111358,125209603,56153341,460658,56613999,4890267,171093,5061360,948396,11110,60360,71470,6081226,4905882,172536,5078418,1340774,156,35933,36089,6455281,-374055,-0.2978532445678561,-392378,-0.3124435187259795,18323,0.014590274158123343
3,10006.0,北海道,,2508580,2799233,5307813,2745228,5112,2750340,240776,4038,244814,34204,76,2055,2131,281149,248428,4098,252526,62651,5,772,777,315954,-34805,-0.651459640198868

In [12]:
# コード 2.11 EXCELから表形式以外のデータを取得する例

import xlrd  #xlrdライブラリをインポート

#入力EXCELファイル名
inFileName='./in/1807nsjin.xls'
wb = xlrd.open_workbook(inFileName)  #Excelファイルオープン
sheet_1 = wb.sheet_by_index(0)  #読み込むシート番号を指定
#7行目の2,3,4列目のセルを取得
row = 6  #0からカウントされるため7行目のこと
cols = [1, 2, 3]  #0からカウントされるため2,3,4列目のこと
for col in cols:
    _type = sheet_1.cell_type(row, col) #セルの型を取得
    _value = sheet_1.cell(row, col).value #セルの値を取得
    #セルの型が3=日付の場合は値の変換が必要
    if _type == 3:
        _value = xlrd.xldate.xldate_as_tuple(_value,0)
    print("行", row, "列", col, "セル形式", _type, "値", _value)

行 6 列 1 セル形式 1 値 北海道
行 6 列 2 セル形式 1 値 札幌市
行 6 列 3 セル形式 2 値 907013.0


In [13]:
# コード 2.12 RDBMSからのデータ取得例

#事前に下記の環境設定が必要。環境がない場合はこのスクリプトは動作しない。
# - ローカルマシン（localhost）にMySQLサーバーがインストールされていて起動済み
# - 'db'という名前のデータベースが作成済み
# - 'username'というユーザーが作成済み（パスワードは'password'）
# - 「住民基本台帳による人口」というテーブルが作成され、データ格納済み

import pymysql  #pymysqlライブラリをインポート

#mysqlサーバーへの接続関連情報を格納する辞書型変数
_mysql={}
_mysql["_host"]='localhost'
_mysql["_user"]='username'
_mysql["_password"]='password'
_mysql["_charset"]='utf8mb4'
_mysql["_db"]='mydb'
try:
    #mysql接続
    connection = pymysql.connect(host=_mysql["_host"],
                                 user=_mysql["_user"],
                                 password=_mysql["_password"],
                                 charset=_mysql["_charset"],
                                 local_infile=True)
    #操作のためのカーソルオープン
    #（引数にpymysql.cursors.DictCursorを指定すると辞書型で出力）
    cur = connection.cursor()
    #使用するデータベースへの接続
    cur.execute("use {_db};".format(**_mysql))

    #実行したいsql文を文字列で生成
    sql='''
    select
        集計年, 団体コード, 人口_計
    from 住民基本台帳による人口
    ;
    '''
    cur.execute(sql)  #sqlの実行
    rows = cur.fetchall()   #sqlの実行結果をすべて取得
    print("件数", len(rows))
    print([title[0] for title in cur.description])
    print(rows[:3])
    #出力内容
    # ['集計年', '団体コード', '人口_計']
    # (('2017', '010006', 5342618),
    #  ('2017', '011002', 1936173),
    #  ('2017', '011011', 229800))

    #操作カーソルとサーバー接続の切断
    cur.close()
    connection.close()
except Exception as e:
    print('MySQLサーバーの環境設定に不備があります。', e)

MySQLサーバーの環境設定に不備があります。 (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)")
